In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from yahoo_fin import options 
from pandas_datareader import data as pdr
from datetime import datetime




In [3]:
from scipy.stats import norm
import numpy as np

def black_scholes_greeks(S, K, T, r, sigma, option_type='call'):
    """
    Calculate Black-Scholes price and Greeks for European options.

    Parameters:
    S : float - current stock price
    K : float - strike price
    T : float - time to expiry (in years)
    r : float - risk-free interest rate (annual)
    sigma : float - volatility of the stock price (annual)
    option_type : str - 'call' for call option, 'put' for put option

    Returns:
    dict: A dictionary containing the option price and Greeks
    """

    # Calculate d1 and d2
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        delta = norm.cdf(d1)
    elif option_type == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        delta = -norm.cdf(-d1)
        theta = (((-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2))) + r * K * np.exp(-r * T)) / 52 # weekly
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")

    # Greeks
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T) / 100
    theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)) / 52 # weekly
    rho = K * T * np.exp(-r * T) * norm.cdf(d2) / 100

    return {
        'price': price,
        'delta': delta,
        'gamma': gamma,
        'vega': vega,
        'theta': theta,
        'rho': rho
    }

# Example usage:
S = 7473.14  # Current stock price 
K = 7500     # Strike price 
T = 87.6/52      # Time to expiry in years 
r = 0.006901    # Risk-free interest rate 
sigma = 0.1771 # Volatility 

# Calculate for a call option
results = black_scholes_greeks(S, K, T, r, sigma, option_type='call')
print(results)

{'price': 711.3871917884749, 'delta': 0.5595778998332818, 'gamma': 0.00022964593442015407, 'vega': 38.263467284908195, 'theta': -4.328406849717006, 'rho': 58.46317523162954}
